[Get Started](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/getting-started-step-1.html)

[Python and DynamoDB](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/GettingStarted.Python.html)

### Create a table 

In [1]:
import boto3
# Get the service resource.
dynamodb = boto3.resource('dynamodb')


#### CLI

```
aws dynamodb create-table \
    --table-name users \
    --attribute-definitions \
        AttributeName=username,AttributeType=S \
        AttributeName=last_name,AttributeType=S \
    --key-schema \
        AttributeName=username,KeyType=HASH \
        AttributeName=last_name,KeyType=RANGE \
    --provisioned-throughput \
            ReadCapacityUnits=2,WriteCapacityUnits=2

aws dynamodb describe-table --table-name users 
```

In [2]:
try:
    # Create the DynamoDB table.
    table = dynamodb.create_table(
        TableName='users',
        KeySchema=[
            {
                'AttributeName': 'username',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'last_name',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'username',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'last_name',
                'AttributeType': 'S'
            },

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 2,
            'WriteCapacityUnits': 2
        }
    )
    
    # Wait until the table exists.
    table.meta.client.get_waiter('table_exists').wait(TableName='users')

    # Print out some data about the table.
    print(table.item_count)

except Exception as e:
    if "Table already exists" in str(e):
        print("Table already exists")
    else:
        raise e

0


In [3]:
table = dynamodb.Table('users')

In [5]:
print(table.creation_date_time)

2020-08-02 07:06:50.643000-04:00


### Insert

In [6]:
table.put_item(
   Item={
        'username': 'john-smith',
        'first_name': 'John',
        'last_name': 'Smith',
        'age': 15,
        'account_type': 'standard_user',
    }
)

{'ResponseMetadata': {'RequestId': 'QNUV3I6F9KLA798D1J5P1JUC47VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 02 Aug 2020 11:13:38 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'QNUV3I6F9KLA798D1J5P1JUC47VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [7]:
table.put_item(
   Item={
        'username': 'janedoe',
        'first_name': 'Jane',
        'last_name': 'Doe',
        'age': 25,
        'account_type': 'standard_user',
    }
)

{'ResponseMetadata': {'RequestId': 'TM1R1VD7N1E4CIA1GQVPPA1917VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 02 Aug 2020 11:13:42 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'TM1R1VD7N1E4CIA1GQVPPA1917VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [25]:
table.put_item(
    Item={
            'account_type': 'standard_user',
            'username': 'johnydan',
            'first_name': 'Josh',
            'last_name': 'Danny',
            'age': 25,
            'address': {
                'road': '1 Main Street',
                'city': 'San Jose',
                'state': 'CA',
                'zipcode': 90001
            }
        }
)



{'ResponseMetadata': {'RequestId': 'D0KUUEU0JMT6QVHMCHUDHIL51RVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 02 Aug 2020 11:41:56 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'D0KUUEU0JMT6QVHMCHUDHIL51RVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [9]:
print(table.item_count)

0


Count number of rows in a table
```
aws dynamodb scan --table-name users --select "COUNT"
```

### read


#### CLI
```
aws dynamodb get-item --consistent-read \
    --table-name users \
    --key '{ "username": {"S": "janedoe"}, "last_name": {"S": "Doe"}}'

```

In [14]:
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
item

{'username': 'janedoe',
 'account_type': 'standard_user',
 'last_name': 'Doe',
 'age': Decimal('20'),
 'first_name': 'Jane'}

### update

In [10]:
table.update_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    },
    UpdateExpression='SET age = :val1',
    ExpressionAttributeValues={
        ':val1': 20
    }
)

{'ResponseMetadata': {'RequestId': '6QGRURB2VUDQ6H99SE0VL1G5PFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 02 Aug 2020 11:23:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6QGRURB2VUDQ6H99SE0VL1G5PFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [13]:
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']

item

{'username': 'janedoe',
 'account_type': 'standard_user',
 'last_name': 'Doe',
 'age': Decimal('20'),
 'first_name': 'Jane'}

### delete

In [15]:
table.delete_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)

{'ResponseMetadata': {'RequestId': 'HEH3U3U446GMMLD6MVRPRQVMPJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 02 Aug 2020 11:25:26 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'HEH3U3U446GMMLD6MVRPRQVMPJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [16]:
response = table.get_item(
    Key={
        'username': 'janedoe',
        'last_name': 'Doe'
    }
)
item = response['Item']
item

KeyError: 'Item'

In [18]:
response = table.get_item(
    Key={
        'username': 'john-smith',
        'last_name': 'Smith'
    }
)
item = response['Item']
item

{'username': 'john-smith',
 'account_type': 'standard_user',
 'last_name': 'Smith',
 'first_name': 'John',
 'age': Decimal('15')}

### Batch Writing

In [19]:
with table.batch_writer() as batch:
    batch.put_item(
        Item={
            'account_type': 'standard_user',
            'username': 'johnydan',
            'first_name': 'Johny',
            'last_name': 'Dan',
            'age': 25,
            'address': {
                'road': '1 Jefferson Street',
                'city': 'Los Angeles',
                'state': 'CA',
                'zipcode': 90001
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'super_user',
            'username': 'janedoering',
            'first_name': 'Jane',
            'last_name': 'Doering',
            'age': 40,
            'address': {
                'road': '2 Washington Avenue',
                'city': 'Seattle',
                'state': 'WA',
                'zipcode': 98109
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'standard_user',
            'username': 'bobsmith',
            'first_name': 'Bob',
            'last_name':  'Smith',
            'age': 18,
            'address': {
                'road': '3 Madison Lane',
                'city': 'Louisville',
                'state': 'KY',
                'zipcode': 40213
            }
        }
    )
    batch.put_item(
        Item={
            'account_type': 'super_user',
            'username': 'alicedoe',
            'first_name': 'Alice',
            'last_name': 'Doe',
            'age': 27,
            'address': {
                'road': '1 Jefferson Street',
                'city': 'Los Angeles',
                'state': 'CA',
                'zipcode': 90001
            }
        }
    )

In [20]:
with table.batch_writer() as batch:
    for i in range(50):
        batch.put_item(
            Item={
                'account_type': 'anonymous',
                'username': 'user' + str(i),
                'first_name': 'unknown',
                'last_name': 'unknown'
            }
        )

### Querying and Scanning

In [22]:
from boto3.dynamodb.conditions import Key, Attr

In [23]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [26]:
response = table.query(
    KeyConditionExpression=Key('username').eq('johnydan')
)
items = response['Items']
pp.pprint(items)

[   {   'account_type': 'standard_user',
        'address': {   'city': 'Los Angeles',
                       'road': '1 Jefferson Street',
                       'state': 'CA',
                       'zipcode': Decimal('90001')},
        'age': Decimal('25'),
        'first_name': 'Johny',
        'last_name': 'Dan',
        'username': 'johnydan'},
    {   'account_type': 'standard_user',
        'address': {   'city': 'San Jose',
                       'road': '1 Main Street',
                       'state': 'CA',
                       'zipcode': Decimal('90001')},
        'age': Decimal('25'),
        'first_name': 'Josh',
        'last_name': 'Danny',
        'username': 'johnydan'}]


In [28]:
response = table.scan(
    FilterExpression=Attr('address.city').eq('San Jose')
)
items = response['Items']
items

[{'username': 'johnydan',
  'address': {'zipcode': Decimal('90001'),
   'state': 'CA',
   'road': '1 Main Street',
   'city': 'San Jose'},
  'account_type': 'standard_user',
  'last_name': 'Danny',
  'first_name': 'Josh',
  'age': Decimal('25')}]

In [29]:
response = table.scan(
    FilterExpression=Attr('first_name').begins_with('J') & Attr('account_type').eq('super_user')
)
items = response['Items']
items

[{'username': 'janedoering',
  'address': {'zipcode': Decimal('98109'),
   'state': 'WA',
   'road': '2 Washington Avenue',
   'city': 'Seattle'},
  'account_type': 'super_user',
  'last_name': 'Doering',
  'first_name': 'Jane',
  'age': Decimal('40')}]

In [30]:
response = table.scan(
    FilterExpression=Key('username').begins_with('user')
)
items = response['Items']
items

[{'username': 'user12',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user11',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user26',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user44',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user1',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user19',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user16',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user8',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user14',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user30

In [34]:
response = table.scan(
    FilterExpression=Key('username').begins_with('user2')
)
items = response['Items']
items

[{'username': 'user26',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user25',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user20',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user29',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user27',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user28',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user24',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user22',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user2',
  'account_type': 'anonymous',
  'last_name': 'unknown',
  'first_name': 'unknown'},
 {'username': 'user2

### Deleting a Table

In [35]:
table.delete()

{'TableDescription': {'TableName': 'users',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 2,
   'WriteCapacityUnits': 2},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:578247465916:table/users',
  'TableId': '886c9956-4f9e-4d11-b83d-63708024c4df'},
 'ResponseMetadata': {'RequestId': 'IS10K80N5R0P6GNAUHEKAOK0KVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 02 Aug 2020 11:52:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '312',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'IS10K80N5R0P6GNAUHEKAOK0KVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3141352462'},
  'RetryAttempts': 0}}